# Get CBA Policy Documents
CBA make donwloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

over come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

### Scraper Code

Notes
1. Only searchs for links in div#wrapper
2. Special code to deal with ASPX/DynamicContent
3. Limited to

In [1]:
from  scraper import WebScraper
from loguru import logger
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import os

class WBC_SGB(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True, ) -> None:
    
        # Stop if depth is exceeded
        if depth and depth <= 0:
            logger.info(f"Bottomed out at {url}")
            return 
        
        path = urlparse(url).path
        extension = os.path.splitext(path)[-1].lower()
        
        # skip already processed urls   
        if path in self.cache:
            return
        
        self.cache.add(path)

        if not url.startswith(self.domain):
            logger.info(f"Skipping domain {url}")
            return

        # Skip unwanted videos etc
        if extension not in [".html", ".aspx"]:
            logger.info(f"Skipping unwanted extention  {url}")
            return
        
        if "Logon.aspx" in path:
            return

        # If aspx find the base html then extract the links and recurse without saving content
        if extension == ".aspx":
            html_urls = self.aspx_link_to_html_link(url)
            for html_url in html_urls:
                self.scrape(html_url, starting_page, self.search_only)
            return
        
        # download all resources except starting page
        if starting_page:
            html_content = self.fetch_content(url)
        else:
            if self.search_only:
                logger.info(f"Saving {url}")           
                html_content = self.fetch_content(url)
            else:
                html_content = self.save_url(url)

        # if we have not yet exceeded the depth find all the links in wrapper and recurse
        if extension == ".html" and depth and depth > 0:
            soup = BeautifulSoup(html_content)
            for link in soup.select('div#wrapper a'):
                if href := link.get('href'):
                    if not urlparse(href).netloc:
                        href = urljoin(self.domain, href)
                    
                    self.scrape(href, depth=depth-1, starting_page=False)
        return

# Download Comm Broker

In [2]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
wbc = WBC_SGB("https://www.commbroker.com.au", 
              cookie="cbpopup=50; s_cc=true; s_nr=1735871648417; s_sq=cfsadviser-cb-prd%3D%2526pid%253Dcb%25253Apopup%25253Aclosebutton%2526pidt%253D1%2526oid%253Dhttps%25253A//www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx%2526ot%253DA; SameSite=None; OpenIdConnect.nonce.xmG6UZmQwcnlBa7D%2F8OllnHO6NJO6eF2h%2FXoYzsXnQA%3D=YTNHWm5fN2drWlFDeE9TUjkwQVkzeDVtZlpWN2NoUjhTMnYxRXV4MWxrTzZpNHRyZV91QkJ1NllmZjFoVnJrRGxqbEpBOU8yR2EzRkVrS2w1dDVwX0tuUHFJSlVCQmE2ck5WaklKSVdKeXY5SW4yZHJKRU5JblFNM19aUlA1SUJrWDdUR1paVGR0anlHVEtFYWR3d2hNSDZ3MWtfZlRYdTNqODNpQ3NFR1NYYU1qbjFNWWFteG0wcXJYYzNrblBHTEE1dHJHeGVVczkwYU51T3lOUVduNnhjajI4; ASP.NET_SessionId=h54uu4g5as1gqxye05lqsf0m; cbid=BJ1Omyn5tbLREe1G769qvPCfbMkAPS4QOo6KdbYUIqfiSgmSBwsgZYd3mtDRS6Q5cDZd8IGXNEYWGcTar8WFGdrE1Cgu-vrDNKYJvQUgh021h7K1hEGTMw1g3Q3ZTqvXZPhHx1EmKw_9ct4UWqPPQo7BZo8uTXwImZ4BtLoDUXOJcvXhtg2pdrt2ZpmAOJzZCVOHBm1DbaOyzO4Tw0c675chbbFLM_rbaknh52I1_0SRBlNYxRsahGW4doG4NxVsWK-ngb5j0ccGzolkv2TBEVFYqAbCUDikfpfBq-rihjjLDNFnwxIt4ZgrarJOjfM_NNigY6ZsUzvyOc8lfiw4WgS7I2qSXOHXx-6IuwB41LSkQ9dBvTiqtYcWoHJpvM-G7IsyK74hf0b7JY1d5eLnAuPGZiyFW5pO9WonTH_WeyGOubn8mLwgdpU-jQ7_-LKbZcDvHWcyMmV6pZP2ua2-k6KSLeURO1l0ea4gl_y4Wwjb2CbutRTmVcTL0f_8bkHmWitWzaxzGfq5Oo5nsexe_oInLDPlTia8SO8d2fdklhojvHQn4Z9QNdPB7pWkDhZ5Rm0Xu6Mnc_w95dHjIAWGrQnhIufACr4hz1hNhXQwdLvSkWCca_BiJTzT6TdmGVj-TLyr3riVOlkGbkkZx07XhpcB2MLKEM0RCxLn-h7OgqBcIZ1QPL2thG5skc_tz6vCco-yVZkDHTVP6KYoSsDwNA1ICMg3TVF-rFTouHhzk3HAeqnt_lzAWNPhD7IeGrMKVzCqB3AubfEs7Xs5HyCJ_ZF2d2O2pr8nnV-mhrw1mXzqsGAzSKfRKH_KzueQ93eIcX3pT6XsduP9tO4qDjWvgeAckv-ZXRwCSud7Wyba717C8moFiBd3jGcreDpzH0Kt762iind5BcLVY4YuMmYtmhhCneEOiDEKKVUPmqt7qyej4RZw1cF5PgYe7p9hzFD0nYywBapfj7jqhwmwSuirjU1m16eTvbnGkChPLeKTKXaYGBW_u9Ftm9cmRpVyF8nYzKQmm5DpRw4NlRr5zsG0FL4f8YCFAvFVPtTu-wW4tjkmUapjK2tWJ3i5o_QT2G4J_U_TxYrf94MTxV0JaiEbxexlPixewpuz_yfJ42S3NZDn6RpkXohDT9kGmvLsplXiVqq-iaOW4X9gTh08JZS4RZ-8gcyWhIiJzlId3gkyHfZhbhS4Wjrig2kRG8GVKxK3r2MOphhFXChBYffZVe_qv0S_3ghQjLpjUckc6aJxU6uhhRquAsEdwhA5DyUITgFr01UWa0RpkUMmePKK3W8ivdnhsH2X7tyUWeZlld10gEYtEVep5QdmJ9EJ2_P1HgK1BZEghM891828QW2nMPLtT0hUDONkLBaxbMPSbqKZ3anicyugr3TqRK4BpJ4WLCH68ZzKxd8tKKFqv4FuFi_3z2gT_cPiF9pLCzNjr5kj7zLAKeWFyiRJHAmInSgqYEqhBLrPoHkw1TOX88ytga3DdCGnbwLkBClVg_9yex6ngdhUDH-pposqBp2QgTTzEnNslmYCrhMxcvq8jxEOH9mUaJ_Vu4guCM9dULNlQK5zeNSiwx63N-YZBb5pL43yc042nz1n0eaaS6b4Y8MtJOUz1oCJVZGMjE5iyvp2czjy_mfivcm4oF1EE19ANQmc25OUlFpSsgTuDpZRPMWyBzKW-pSShRrKE79xgASCo-qgQ3dZAvkydzyEp53-iil9AsKgJCEtIfAziNSGBgxpjR9KvyMOtGHADli57GIO88OUvk2X8oGUg_juzS1s3Iz5TGfhSIfygyp93vdm2dzDNcnYyfY3z4yMTbh27ofMAUOPWlxpMQVjQAbdfez2FCkGIjdDWf8Ow6FMV_oXYLas5WK-YAuJXGIBZxlmFtnSAAOAow_QrXlsHoNvTNX4lEGTAnUWBbDz2W8MUT29zvWFCq2rsPXguXootILHMFQGzDLQ0SpY8CZ-ESX2IGjP1WEDa3VXJUfhDVQ_im1dETQTanVKCUzR6T4H8qzRK0o6YWhhcP5D5enLStN3ZR3jzI50aut8KVVnOnXQAlZ9pAl1rTGekUbO5o1YG2PlgDuTWzJMJXSQZXP978XEeztxFD8PsqqJMPUqzRWrtHeV9HchcQWQ2EagFkQInvcvyFQsZXF2NpQQNNiuOybiT_gPFTbtp9q9sn7qhRnYER6jUPxg7hqWqPHo7lddWXPjk-kN0GG0JFKiqmRE7xgKQqobz6gY_lC1su_S8HZSnrJ3A13xB9mA0DzJnQYjkQuxLN-_cRF-cMaqhZ4SGamr5tEagNrQyPge6HFVvMlVoCON7jpM3PjqaNIJl3NdVe9YhohVmRh82mp-K3Pg607s5KZsEihWi5dp_YpeEMi3YMUW-Sh8l6diysm6seE_i2c9I5VEzYiiEfpUvFpZAcXdbFujhTUdu4pgvb9UPu0mE9opAC3xfUMi3od24X9Hgh8SLCPxdR_eUzlRi0DQ_cf9RpllLEN20PfSxtCU8tHyywA_uRBASeawR3TklzsckeBkrFVbpXGMGgCISDD0n1FLb-IRD_w2TRtrg03YXkkWD7MsmAkmcuh88K0JwN0JbwPv1fsjJAexkz7he0YIiBmuqlmIHRpYDs811Rg_Hrc-Knj8N1Z8g4hol4I2AzvwXdD2pS1EoWdmZ6yQOsvjaU_PyNOG60NKNvNvSQgehRbe5XENDcd2afgzh2TyLo0P43oQvzJSV_iqJ9ljsvNsK68yayub3OAF1F8V13Y5lzJMKn4YW-Ky6nYhyp6eDoGE5BHbDBVqWDqBiRAIqyAjG-HHqtETlUCMBGKVfM-GYBTx18Y8ZPA9PGnCCZjMpbsM4wU7AXEH5KR5BweAkkCeocy84nDENleD_au-g7Jh2nEUL87K_OwbSnZF8yKIgdPHAV-tE2Uw5mRAUhAcfmy6sn8w8okbeaZ1theO9DpDdwjCF5zRdX19--ZM-nInXJwuUPzZbYolgalVgbMlHMSSPOeUiB89p7keI-dAwLyR4qnlO-ChGwObethPmBWpWAQQSR_mFI-g-t0ZCDiBXDmNlYVGv0f95FKaVUskoYS-GgodxLybU62ezI4O_H2NAkZr9YivkC6bpohImw6WUd_ugKoc78hgs-zeJb000Lv2BFhOiS_4khqmw0GMUpYSUnW1C9vaQggyjq7OLju0Kbma9GH630H5xBq5z4joSwHD4ZT8dXzaf4tRqvH4ilZ7I9B9GPVoDQwe1D-8kWBsA74p-ug_zPtfF_leYuwILlqkba_T9t99mFF--eiNPNSHNHScMa-lJmnnTrANePSviw_p9U-XpsK9hac1wSWZhDXSUmVuSre6VnXuN-8m9xwzcB52wt1ehrEP97YMgLWQIDFDs0tajzHXJXaNCJny7Fj4iIiKgS7To3j2Bdce9byHk6ks7Wq3_2UA6cTLXxj-_cBrjWwV4W7xL9z9yTnoSZxiGA; bm_mi=11626FB2DD8017BCB7AE009C65792477~YAAQXgNiaG1axSmUAQAAJkxnLhovykxNT7ZOucp2XhL07cSrcngqb6LNB/cGZl3CbklJPGEvh8LGLnkmon8aRUIII8MFP3r6YEUIvc4I6CqcECL0uf/4iZkiI+OzfgkMeEH8mYcH86gLcK6dXKtwFeFhUp0agtbtlig9Uls/1+dY6hZB/9Cd7F5Ej4b63vzNfakahsyKEW9zUxdlULtL3uWtNGarbRCcNHVN4XSUdLqXRnPzeQYAuzN+QU7boDcViMoaghVNlZ4y5Q8tpC2T+aTBXdWN9pxm0zr0pVV4AFirwiTROV/i0Wr+nlFSfEIbH/PhPjxKSjCTmXTBnqytbI45Pnm9r2npW50H7kn8iMq8GVbzveVQkDEVUzb70A==~1; ADRUM_BTa=R:90|g:8f537302-6bef-422d-a195-3ecdfd4fa89a|n:cba-prod_0e85eb6c-1d2c-4a35-a7e2-92db3c234685; htsm=exp=1735947113~acl=/*~hmac=5d3f2196cd7b2399396aa15c968eaf9130fdd2f0a3439e2a989ef1ed158d045e; ak_bmsc=A762ACA4645DF86374E2BB8FA1286131~000000000000000000000000000000~YAAQXgNiaNxbxSmUAQAArFxnLhrQbbsEBFA7X+JhQnDJigJL6ADGsPMRTwk4WdPdbAHXdEUQFWDF7GHjVkXdNIcYwEFo4x+4LS/BuyVSnGXqlpSJXVCS+OZ7E8ihVPdBivr00nOK4gZB0m6k7dp8Gczh14ed/uCjMzPfVEh0+pExv9kNNQyzIpFv8QYi3aoZzmB/5HpmIy/ZzFGBj4nHDH/xqo/gRgatpXplfyhpJ8fxP8tu0QeqnqrnDVh8Bgt7QfHw0jFpHv4fJ8MLmGYLFPW8tcblgFiai+MswGsqnNgsd4NPNkyLR7iWG/ajQfgmuSitTdZHixuy9iDdKw2YqSkUW2lshqZ8hhDwMgXbU7jfk1rwj9bt3+ayyHX+v43Eiq5091L3vuZG49KY4NXCRHtXKXu3AbYliwlFk5IiLn4l4DksXkf7V7PGR4CD+qX+DAbVY16FIz0wfXOWfYn4oc4F5A6Wo7MXnZmov1NiYG8S83GwyCiFP/8Ksa9xkou1ahQrSR9J/GSjc1qpdGyxwyHV+uxtgh78iXv+QHs=; bm_sz=625B4D99506E356293CEA4B0D823681A~YAAQXgNiaN5bxSmUAQAArFxnLhqPSTXZYPGGc7Kb509U0FSpPkZe3V8GZfwfTaDZOCdOEnjn3qEQSHQabQrp8++6uFpv8SF28mjVlgwMFzAk0UFkzWfuMmnbjLJQ+LY1aPBeSBxzk4PTuR7SnRBBpxfwS7ZvDTiOecjktTUAczUFgLHxXBpcJ7iw93oWYt1FrXmYcnDXUewI1TZz655I5wJuRz/0+DW6kkL8raDX2A1NgxriXwWxDpKSb7nlnBHXuutmweEB4F0pbzQKhukGngKMdY1YJVMpJsFsRQiXn/FI+WvkWGB38p4Hb3h7kblokJodn7J1irVqj2W8k4regnapXB3yTePBaNwfQT8+w6KA/3UO5ZZnSyobiIfRmTur6BsNAocj2pC0Q4rQ9o/4oEf+yLz1KF4yAAhBH/cp80Lp3b5TigulajoJuK/8~3293491~3294788; bm_sv=92CB8F6BB309DC8793972D735D95BD89~YAAQRQNiaOUABCmUAQAA4I1nLhrt4/Zo3EV0DJe3maeNblMYv2M2AHutak1EyJzbC4vi0dPDih1UxjN7+jPClt2C2dD8mcZhUN9JotNrTpp3KX/GfPO+rrVprTqNLGizmkG+n4PJuN1C2Tm08Df3uJzF8rV2IueHGMvOEFYr43FX23AvV6oxve/6ukKTyFGYWQXaxYOBUBP0Wtb9OIuqDkxXY+S1dsITvSxxj7pm5I2TF9P7c3oedDsAKWvP7F4sxdhFO9ZMYg==~1; _abck=64879BC9507C9570BF71DAA593F45962~-1~YAAQRQNiaBUBBCmUAQAANpBnLg2eJmUrLc9orxDHDR3uLuqBYoE8Eph/AbqaCzwRbPzf0Kfs5R1YfP5PGOnYyaijGwOZA1TVv6NnJay5+39WpGZiCoUVMewLf1/RTttFGHPa5Fb7OxhQp6P9PippYZWEm6W3KGad0nDiV3QelvM+cz0BJ2tNHlqpEnNUNaWf8nfw/WKXkjmuexFcx8ZQjgferPrW9/9iUEDJ/TXUkM/XvpKvIVJGXoN/ALCajDxp/5AGzrn1q1a2WUH73VlQqQ60362A7KG2S7nyz2BJ1yt+1vcziU1RXcXx5dvM4Q7MfIJ1fs06xi8/L1NISD82Kj18h7uosXrYgEEwPaZinKZpx/CQoO06qUNBofBd/Bld1h+mfKn3N0bQp89jL30JhmnPTDUd+9b5NuLT9d2uK0dHZkREZJ+fEUAfYIuNPZb1zkgwazQUmqpBs9X9IFopcKRdn9o29tR/t8ZKjipF4npUjCWO355VM2EamrtRe4riivP/ZKtgqFT4y7AgT/g=~-1~-1~-1", 
              download_path= directory_path, 
              search_only=False)
wbc.scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", depth=3)
wbc.scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx", depth=3)

print(len(wbc.cache))

2025-01-04 07:20:53.969 | INFO     | scraper:save_url:47 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/default.html
2025-01-04 07:20:53.978 | INFO     | __main__:scrape:29 - Skipping domain https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-01-04 07:20:54.331 | INFO     | scraper:save_url:47 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/acceptable-excluded-purposes.html
2025-01-04 07:20:54.737 | INFO     | scraper:save_url:47 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/eligible-excluded-borrowers.html
2025-01-04 07:20:55.181 | INFO     | scraper:save_url:47 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/regulatory-requirements/acceptable-identification-documents.html
2025-01-04 07:20:55.496 | INFO     | scraper:save_url:47 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/

169
